# Geographical Indexing Explained

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Geohash level 3 (0.7° by 0.7°)

https://stackoverflow.com/a/39540339/7195218

In [ ]:
x = np.linspace(-90, 90, 100)
latitude distance